## Параллельный расчет с Dask
https://dask.org/

In [69]:
# Функция для параллельного расчета
import numpy as np
import random

def foo(geom_kr, geom_op, geom_korp):
    from NAVIver2 import Target, Rocket
    import time
        
    targel = Target(3000, 3500, 320)
    rockk = Rocket(targel.y, targel.x, geom_kr, geom_op, geom_korp)
    # time.sleep(np.random.random())
    return rockk.navigation(targel)

def da_fit(rest):
    maxis = 0
    summF = 0
    for i in range(len(rest)):
        summF += rest[i]
        if maxis <= rest[i]:
            maxis = rest[i]
    ans = np.zeros(len(rest))
    for i in range(len(rest)):
        ans[i] = rest[i] / summF
    return ans

def crosss(pipi):
    krils, opers, korps, kri1, op1, korp1 = [], [], [], [], [], []
    for i in range(len(pipi)):
        krils.append(pipi[i][1])
        opers.append(pipi[i][2])
        korps.append(pipi[i][3])
    for i in range(len(krils[0])):
        kri1.append(krils[random.randint(0,1)][i])
        op1.append(opers[random.randint(0,1)][i])
    for i in range(len(korps[0])):
        korp1.append(korps[random.randint(0,1)][i])
    chichi = [0, kri1, op1, korp1]
    return chichi
        

def da_new(croms):
    croms.sort()
    parent = croms[int(len(results)/2):]
    child = []
    # лучший родитель
    child.append(parent[len(parent) - 1])
    # дети лучших родителей    
    for i in range(int(len(parent) * 2) - 2):
        child.append(crosss(parent))
    
    # один рандомный мутант
    
    params_kr1 = np.zeros(5)
    params_op1 = np.zeros(5)
    params_kr1[0] = random.uniform(0.04, 0.115)
    params_kr1[1] = random.uniform(0.002336, 0.010564)
    params_kr1[2] = random.uniform(0.065, 0.105)
    params_kr1[4] = random.uniform(0, 0.032)
    
    params_op1[0] = random.uniform(0.05, 0.18)
    params_op1[1] = random.uniform(0.002, 0.0052)
    params_op1[2] = random.uniform(0.02, 0.034)
    params_op1[4] = random.uniform(0.328, 0.0356) 
    
    params_krop1 = np.array([0.0974, 0.050, 0.160])
    mutant = [0, params_kr1, params_op1, params_krop1]
    child.append(mutant)
    
    return child

    

In [2]:
# 

from distributed import Client, LocalCluster

cl = Client(LocalCluster(n_workers=16, threads_per_worker=1))
cl

Client Scheduler: tcp://127.0.0.1:56013 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 16 Cores: 16 Memory: 15.88 GiB


In [3]:
# заполнение параметров

# параметры крыльев
params_kr = np.array([[0.096, 0.008912, 0.10568, 0, 0.015], 
                     [0.04, 0.0022, 0.065, 0, 0.015],
                     [0.115, 0.010564, 0.105, 0, 0.015],
                     [0.098, 0.00917336, 0.10568, 0, 0.015]])

# ограничение по длине: 0.04...0.115
# ограничение по площади: 0.002336 ... 0.010564
# ограничение по бортовой хорде: 0.065 ... 0.105
# ограничение по расстоянию до кормового среза: 0.0 ... 0.032
# 

# параметры рулей
params_op = np.array([[0.17824, 0.0055380742, 0.03214, 0, 0.35], 
                      [0.005, 0.002, 0.02, 0, 0.35], 
                      [0.18, 0.0052, 0.034, 0, 0.35],
                      [0.12576, 0.0038426, 0.034, 0, 0.35]])

# ограничение по длине: 0.05...0.18
# ограничение по площади: 0.002 ... 0.0052
# ограничение по корневой хорде: 0.02 ... 0.034
# ограничение по расстоянию от носика: 0.328 ... 0.0356
# 

# параметры корпуса
params_korp = np.array([[0.0974, 0.050, 0.160], 
                        [0.0974, 0.050, 0.160], 
                        [0.0974, 0.050, 0.160],
                        [0.0974, 0.050, 0.160]])
# params_x = ([3000, 3000, 3000, 3000])
# params_y = ([3500, 3500, 3500, 3500])
# params_v = np.array([320, 320, 320, 320])

In [4]:
futures = cl.map(foo, params_kr, params_op, params_korp, pure=False)

In [5]:
futures

[<Future: pending, key: foo-94bdb49e-f73b-4c5b-8c51-c5d374ca530c-0>,
 <Future: pending, key: foo-94bdb49e-f73b-4c5b-8c51-c5d374ca530c-1>,
 <Future: pending, key: foo-94bdb49e-f73b-4c5b-8c51-c5d374ca530c-2>,
 <Future: pending, key: foo-94bdb49e-f73b-4c5b-8c51-c5d374ca530c-3>]

In [6]:
results = cl.gather(futures)

In [7]:
results

[0.9668007702402822,
 0.1852347775599727,
 0.9201526105472436,
 1.1525261449633764]

In [8]:
dab = da_fit(results)


In [9]:
dab

array([0.29980974, 0.05744223, 0.28534392, 0.35740411])

In [10]:
summs = 0
for i in range(len(results)):
    summs += dab[i]
kris = summs / len(results)



In [37]:
croms = []
for i in range(len(results)):
    croms.append([dab[i], params_kr[i], params_op[i], params_korp[i]])
croms

[[0.29980974415241984,
  array([0.096   , 0.008912, 0.10568 , 0.      , 0.015   ]),
  array([0.17824   , 0.00553807, 0.03214   , 0.        , 0.35      ]),
  array([0.0974, 0.05  , 0.16  ])],
 [0.05744222902778974,
  array([0.04  , 0.0022, 0.065 , 0.    , 0.015 ]),
  array([0.005, 0.002, 0.02 , 0.   , 0.35 ]),
  array([0.0974, 0.05  , 0.16  ])],
 [0.2853439170107273,
  array([0.115   , 0.010564, 0.105   , 0.      , 0.015   ]),
  array([0.18  , 0.0052, 0.034 , 0.    , 0.35  ]),
  array([0.0974, 0.05  , 0.16  ])],
 [0.3574041098090631,
  array([0.098     , 0.00917336, 0.10568   , 0.        , 0.015     ]),
  array([0.12576  , 0.0038426, 0.034    , 0.       , 0.35     ]),
  array([0.0974, 0.05  , 0.16  ])]]

In [38]:
croms.sort()
croms

[[0.05744222902778974,
  array([0.04  , 0.0022, 0.065 , 0.    , 0.015 ]),
  array([0.005, 0.002, 0.02 , 0.   , 0.35 ]),
  array([0.0974, 0.05  , 0.16  ])],
 [0.2853439170107273,
  array([0.115   , 0.010564, 0.105   , 0.      , 0.015   ]),
  array([0.18  , 0.0052, 0.034 , 0.    , 0.35  ]),
  array([0.0974, 0.05  , 0.16  ])],
 [0.29980974415241984,
  array([0.096   , 0.008912, 0.10568 , 0.      , 0.015   ]),
  array([0.17824   , 0.00553807, 0.03214   , 0.        , 0.35      ]),
  array([0.0974, 0.05  , 0.16  ])],
 [0.3574041098090631,
  array([0.098     , 0.00917336, 0.10568   , 0.        , 0.015     ]),
  array([0.12576  , 0.0038426, 0.034    , 0.       , 0.35     ]),
  array([0.0974, 0.05  , 0.16  ])]]

In [39]:
print(croms[int(len(results)/2):])

[[0.29980974415241984, array([0.096   , 0.008912, 0.10568 , 0.      , 0.015   ]), array([0.17824   , 0.00553807, 0.03214   , 0.        , 0.35      ]), array([0.0974, 0.05  , 0.16  ])], [0.3574041098090631, array([0.098     , 0.00917336, 0.10568   , 0.        , 0.015     ]), array([0.12576  , 0.0038426, 0.034    , 0.       , 0.35     ]), array([0.0974, 0.05  , 0.16  ])]]


In [40]:
print(croms[0])

[0.05744222902778974, array([0.04  , 0.0022, 0.065 , 0.    , 0.015 ]), array([0.005, 0.002, 0.02 , 0.   , 0.35 ]), array([0.0974, 0.05  , 0.16  ])]


In [70]:
chicha = da_new(croms)
print(chicha)

[[0.3574041098090631, array([0.098     , 0.00917336, 0.10568   , 0.        , 0.015     ]), array([0.12576  , 0.0038426, 0.034    , 0.       , 0.35     ]), array([0.0974, 0.05  , 0.16  ])], [0, [0.098, 0.008912, 0.10568, 0.0, 0.015], [0.17824, 0.0055380742, 0.03214, 0.0, 0.35], [0.0974, 0.05, 0.16]], [0, [0.096, 0.008912, 0.10568, 0.0, 0.015], [0.12576, 0.0055380742, 0.034, 0.0, 0.35], [0.0974, 0.05, 0.16]], [0, array([0.05367478, 0.0093764 , 0.09995881, 0.        , 0.00850575]), array([0.12860549, 0.00455378, 0.02075837, 0.        , 0.23160194]), array([0.0974, 0.05  , 0.16  ])]]


In [71]:
for i in range(len(chicha)):
    print(chicha[i])
    print()

[0.3574041098090631, array([0.098     , 0.00917336, 0.10568   , 0.        , 0.015     ]), array([0.12576  , 0.0038426, 0.034    , 0.       , 0.35     ]), array([0.0974, 0.05  , 0.16  ])]

[0, [0.098, 0.008912, 0.10568, 0.0, 0.015], [0.17824, 0.0055380742, 0.03214, 0.0, 0.35], [0.0974, 0.05, 0.16]]

[0, [0.096, 0.008912, 0.10568, 0.0, 0.015], [0.12576, 0.0055380742, 0.034, 0.0, 0.35], [0.0974, 0.05, 0.16]]

[0, array([0.05367478, 0.0093764 , 0.09995881, 0.        , 0.00850575]), array([0.12860549, 0.00455378, 0.02075837, 0.        , 0.23160194]), array([0.0974, 0.05  , 0.16  ])]



In [73]:
params_kr, params_op, params_korp = [], [], []
for i in range(len(chicha)):
    params_kr.append(chicha[i][1])
    params_op.append(chicha[i][2])
    params_korp.append(chicha[i][3])

futures = cl.map(foo, params_kr, params_op, params_korp, pure=False)
cl.gather(futures)

[1.1525261449633764, 0.9659235304809753, 0.9692540899055379, 0.811317878915653]